# Impot libraries #

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import sqlite3

# Define files to read #

## Parcelles ##

In [2]:
file_NPCS_PARV = "../data/shapefiles/NPCS_CAD_TPR_PARV_S.shp"
file_MOVD_PARV= "../data/shapefiles/MOVD_CAD_TPR_PARV_S.shp"
#file_NPCS_DDP = "../data/shapefiles/NPCS_CAD_TPR_DDP_S.shp"
#file_MOVD_DDP = "../data/shapefiles/MOVD_CAD_TPR_DDP_S.shp"

## Other data ##

In [3]:
file_communes_vd = '../data/communes_vd.csv'

# DB connexion #

In [4]:
db_file = '../data/APMK.sqlite'
con = sqlite3.connect(db_file)

# Load parcelles data #

In [5]:
parcelles_NPCS_PARV = gpd.read_file(file_NPCS_PARV)
parcelles_MOVD_PARV = gpd.read_file(file_MOVD_PARV)
#parcelles_NPCS_DDP = gpd.read_file(file_NPCS_DDP)
#parcelles_MOVD_DDP = gpd.read_file(file_MOVD_DDP)

## Merge parcelles dataframe ##

In [6]:
parcelles_shape = pd.concat([parcelles_NPCS_PARV, parcelles_MOVD_PARV])
#parcelles_shape2 = pd.concat([parcelles_NPCS_DDP, parcelles_MOVD_DDP])
#parcelles_shape2.insert(1, 'IDEX2000', "")
#parcelles_shape = pd.concat([parcelles_shape1, parcelles_shape2])
parcelles_shape.reset_index(drop=True, inplace=True)

## load no OFS communes ##

In [7]:
no_ofs_communes = pd.read_sql_query("SELECT no_commune as no_ofs, nom_commune as commune from communes", con)

## load potential land usage ##

In [8]:
land_usage = pd.read_sql_query("SELECT nature FROM types_nature WHERE potentiel = 1", con)

## load public parcelles ##

In [9]:
fields = ['no_commune', 'nom_commune', 'immeuble', 'surface', 'proprietaire', 'type_nature', 'surface_nature']
listToStr = ', '.join([str(elem) for elem in fields])
parcelles_conf = pd.read_sql_query("SELECT DISTINCT " + listToStr + " FROM APMK", con)

# Find usable parcelles #

In [10]:
parcelles_pot = parcelles_conf[parcelles_conf['type_nature'].isin(land_usage['nature'])]

# Import data of communes of Vaud #

In [11]:
communes_vd_data = pd.read_csv(file_communes_vd)
# cleanup
communes_vd_data = communes_vd_data.drop(['GEOMETRY'], axis=1)
communes_vd_data = communes_vd_data[['ogc_fid', 'no_com_can', 'no_com_fed', 'nom_min', 'perimetre']]

## Add no commune Vaud ##

In [12]:
communes_no = communes_vd_data[['no_com_fed', 'no_com_can', ]]
communes_no = communes_no.rename(columns={"no_com_fed": "no_commune"})
parcelles_pot = pd.merge(parcelles_pot, communes_no, how='left', on='no_commune')

## build IDEX2000 value ##

In [13]:
parcelles_pot['IDEX2000'] = parcelles_pot['no_com_can'].map('{:0>3}'.format) + parcelles_pot['immeuble'].map('{:0>6}'.format)

In [14]:
parcelles_shape['IDEX2000'] = parcelles_shape['IDEX2000'].map('{:0>9}'.format)

## build ficheID ##

In [15]:
parcelles_pot['fiche_id'] = parcelles_pot['no_com_can'].map(str) + "_" + parcelles_pot['immeuble'].map(str)

## Add new column 'code' ##
**code** : 
1 = commune
2 = confédération
3 = état
4 = autre

In [16]:
parcelles_pot['code'] = None

In [17]:
for index, row in parcelles_pot.iterrows():
    if row['nom_commune'] == row['proprietaire'].split(" ", 1)[0]:
        parcelles_pot.at[index,'code'] = 1
    elif row['proprietaire'].split(" ", 1)[0] == "Confédération":
        parcelles_pot.at[index,'code'] = 2
    elif row['proprietaire'].split(" ", 1)[0] == "Etat":
        parcelles_pot.at[index,'code'] = 3
    else:
        parcelles_pot.at[index,'code'] = 4

# New dataframe #

In [18]:
parcelles_pot_geom_data = parcelles_pot[['no_commune', 'no_com_can', 'IDEX2000', 'fiche_id', 'nom_commune', 'immeuble', 'code']]

## Drop duplicate rows and reindex ##

In [19]:
parcelles_pot_geom_data = parcelles_pot_geom_data.drop_duplicates()
parcelles_pot_geom_data.reset_index(drop=True, inplace=True)

In [20]:
parcelles_pot_geom = gpd.GeoDataFrame(parcelles_pot_geom_data)
parcelles_pot_geom['geometry'] = None
parcelles_pot_geom = parcelles_pot_geom.set_crs(crs=2056)
parcelles_pot_geom = parcelles_pot_geom.drop(['geometry'], axis=1)

parcelles_geom = parcelles_shape[['IDEX2000', 'geometry']]
parcelles_pot_geom = pd.merge(parcelles_pot_geom, parcelles_geom, how='left', on='IDEX2000')

In [ ]:
import contextily as cx
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,10))
parcelles_pot_geom.plot(ax=ax, cmap='rainbow', column='code', legend=True)
cx.add_basemap(ax=ax, crs=2056, source=cx.providers.Stamen.TonerLite)

In [ ]:
#parcelles_pot_geom['nom_commune' == 'Apples']
mex  = parcelles_pot_geom.loc[parcelles_pot_geom['nom_commune'] == 'Mex']
#mex  = parcelles_pot_geom.loc[parcelles_pot_geom['fiche_id'] == '138_36']

In [ ]:
mex = mex[['nom_commune', 'code', 'fiche_id', 'geometry']]

In [ ]:
mex['cent'] = mex.centroid

In [ ]:
Apples